In [97]:
%pylab inline
from sklearn import \
        datasets, \
        linear_model, \
        cross_validation, \
        naive_bayes, \
        preprocessing, \
        svm

import pandas as pd
import matplotlib.pyplot as plt
import json
import csv

from pprint import pprint
from sklearn.cross_validation import KFold

Populating the interactive namespace from numpy and matplotlib


In [14]:
df_train = pd.read_json('train.json')
df_test = pd.read_json('test.json')
df_test.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [19]:
print "number of dishes: " + str(len(df_train))

number of dishes: 39774


In [30]:
list_ingredients = list([])

for d in df_train['ingredients']:
    s = ','.join(d).strip()
    for ss in s.split(','):
        if ss not in list_ingredients:
            list_ingredients.append(ss)

print "number of unique ingredients: " + str(len(list_ingredients))

number of unique ingredients: 6725


In [24]:
list_ingredients.sort()

print list_ingredients

[u'', u' 1 inch thick', u' cook and drain', u' cooked and drained', u' crisp-cooked and crumbled', u' crush', u' cut french into loaf', u' cut into', u' cut into italian loaf', u' cut into serving pieces', u' drain', u' drain and flake', u' drained and chopped', u' fine chop', u' peel and devein', u' rins and drain', u' rins and pat dry', u' skinless chicken breast', u' slice', u' sliced', u' soften', u' split and toasted', u' thaw', u' thaw and drain', u' thawed and squeezed dry', u' thawed and undiluted', u' undrain', u' undrain and chop', u' well scrub', u' well scrubbed', u' wine', u'(    oz.) tomato sauce', u'(   oz.) tomato paste', u'(10 oz.) frozen chopped spinach', u'(14 oz.) sweetened condensed milk', u'(14.5 oz.) diced tomatoes', u'(15 oz.) refried beans', u'1% low-fat buttermilk', u'1% low-fat chocolate milk', u'1% low-fat cottage cheese', u'1% low-fat milk', u'2 1/2 to 3 lb. chicken', u'2% low fat cheddar chees', u'2% low-fat cottage cheese', u'2% lowfat greek yogurt', u'2%

In [35]:
list_cuisines = list([])

for c in df_train['cuisine']:
    if c not in list_cuisines:
        list_cuisines.append(c)

print "number of cuisines: " + str(len(list_cuisines))

number of cuisines: 20


In [36]:
print list_cuisines

[u'greek', u'southern_us', u'filipino', u'indian', u'jamaican', u'spanish', u'italian', u'mexican', u'chinese', u'british', u'thai', u'vietnamese', u'cajun_creole', u'brazilian', u'french', u'japanese', u'irish', u'korean', u'moroccan', u'russian']


In [47]:
# Feature Matrix - Training Data

X1 = np.zeros((len(df_train), len(list_ingredients)))
Y1 = np.zeros((len(df_train)))

for i, d in enumerate(df_train['ingredients']):
    s = ','.join(d).strip()
    for ingredient in s.split(','):
        X1[i][list_ingredients.index(ingredient)] = 1
    Y1[i] = list_cuisines.index(df_train['cuisine'][i])
    
print "shape of X1: " + str(X1.shape)

shape of X1: (39774, 6725)


In [126]:
# Feature Matrix - Test Data

X2 = np.zeros((len(df_test['ingredients']), len(list_ingredients)))

for i, d in enumerate(df_test['ingredients']):
    s = ','.join(d).strip()
    for ingredient in s.split(','):
        if ingredient in list_ingredients:
            X2[i][list_ingredients.index(ingredient)] = 1
            
print "shape of X2: " + str(X2.shape)

shape of X2: (9944, 6725)


In [131]:
for i in range(len(X2)):
    for j in range(len(X2[i])):
        if X2[i][j] == 1:
            print i,j

0 15
0 18
0 53
0 115
0 229
0 495
1 18
1 47
1 50
1 99
1 369
1 580
1 598
1 756
1 1353
1 1753
1 1985
2 35
2 54
2 285
2 1732
2 2707
2 2830
3 19
3 25
3 29
3 42
3 58
3 105
3 115
3 184
3 206
3 210
3 215
3 252
3 330
3 350
3 626
3 749
3 1252
3 1387
3 2340
3 3022
3 5599
4 11
4 13
4 29
4 78
4 335
4 392
4 700
4 1077
5 11
5 37
5 53
5 99
5 115
5 191
5 229
5 312
5 3127
6 525
6 4547
6 6271
7 3
7 19
7 26
7 47
7 51
7 60
7 93
7 99
7 140
7 161
7 204
7 228
7 382
7 800
7 1072
7 1085
8 19
8 29
8 41
8 49
8 75
8 82
8 115
8 140
8 244
8 447
8 570
8 639
8 682
8 697
8 1128
8 2166
8 2175
9 15
9 27
9 49
9 50
9 443
9 495
9 583
9 1587
9 1634
9 1712
9 1801
9 2310
9 2458
9 2863
9 3901
9 3902
10 3
10 11
10 54
10 141
10 261
10 305
10 307
10 552
10 602
10 665
10 2295
11 8
11 18
11 27
11 29
11 34
11 35
11 49
11 82
11 115
11 190
11 346
11 527
11 531
11 823
11 1315
11 4711
12 41
12 51
12 75
12 256
12 275
12 280
12 727
12 825
13 11
13 169
13 230
13 261
13 665
13 714
13 1223
14 11
14 15
14 229
14 415
14 507
14 2256
15 11
15 15


In [82]:
# Cross-validation - Naive Bayes Gaussian prior

cv = KFold(len(df_train))
for trains, tests in cv:
    X1_cv = []
    Y1_cv = []
    X2_cv = []
    Y2_cv = []
    for i in trains:
        X1_cv.append(X1[i])
        Y1_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    for i in tests:
        X2_cv.append(X1[i])
        Y2_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    classifier_ga = naive_bayes.GaussianNB()
    Y2_pred = classifier_ga.fit(X1_cv, Y1_cv).predict(X2_cv)
    n_correct = 0;
    for i in range(0, len(Y2_cv)):
        if Y2_cv[i] == Y2_pred[i]:
            n_correct += 1
    print 'Gaussian ' + str(tests) + ': ' + str(n_correct) + " - " + str(100*float(n_correct)/len(Y2_cv)) + "%"

Gaussian [    0     1     2 ..., 13255 13256 13257]: 5025 - 37.9016442902%
Gaussian [13258 13259 13260 ..., 26513 26514 26515]: 5071 - 38.2486046161%
Gaussian [26516 26517 26518 ..., 39771 39772 39773]: 5007 - 37.7658772062%


In [83]:
# Cross-validation: Naive Bayes Bernoulli prior

cv = KFold(len(df_train))
for trains, tests in cv:
    X1_cv = []
    Y1_cv = []
    X2_cv = []
    Y2_cv = []
    for i in trains:
        X1_cv.append(X1[i])
        Y1_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    for i in tests:
        X2_cv.append(X1[i])
        Y2_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    classifier_be = naive_bayes.BernoulliNB()
    Y2_pred = classifier_be.fit(X1_cv, Y1_cv).predict(X2_cv)
    n_correct = 0;
    for i in range(0, len(Y2_cv)):
        if Y2_cv[i] == Y2_pred[i]:
            n_correct += 1
    print 'Bernoulli ' + str(tests) + ': ' + str(n_correct) + " - " + str(100*float(n_correct)/len(Y2_cv)) + "%"

Bernoulli [    0     1     2 ..., 13255 13256 13257]: 9072 - 68.4266103485%
Bernoulli [13258 13259 13260 ..., 26513 26514 26515]: 9013 - 67.9815960175%
Bernoulli [26516 26517 26518 ..., 39771 39772 39773]: 9104 - 68.6679740534%


In [84]:
# Cross-validation: Logistic Regression

cv = KFold(len(df_train))
for trains, tests in cv:
    X1_cv = []
    Y1_cv = []
    X2_cv = []
    Y2_cv = []
    for i in trains:
        X1_cv.append(X1[i])
        Y1_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    for i in tests:
        X2_cv.append(X1[i])
        Y2_cv.append(list_cuisines.index(df_train['cuisine'][i]))
    classifier_lr = linear_model.LogisticRegression()
    Y2_pred = classifier_lr.fit(X1_cv, Y1_cv).predict(X2_cv)
    n_correct = 0;
    for i in range(0, len(Y2_cv)):
        if Y2_cv[i] == Y2_pred[i]:
            n_correct += 1
    print 'Logistic Regression ' + str(tests) + ': ' + str(n_correct) + " - " + str(100*float(n_correct)/len(Y2_cv)) + "%"

Logistic Regression [    0     1     2 ..., 13255 13256 13257]: 10286 - 77.5833459044%
Logistic Regression [13258 13259 13260 ..., 26513 26514 26515]: 10237 - 77.2137577312%
Logistic Regression [26516 26517 26518 ..., 39771 39772 39773]: 10324 - 77.869965304%


In [87]:
# Logistic Regression on all data

Y_pred = classifier_lr.fit(X1, Y1).predict(X2)

print 'classification complete'

classification complete


In [90]:
print Y_pred[0:100]

[  9.   1.   6.  12.   6.   1.   5.   8.   7.   9.   6.   0.   3.   6.   1.
  14.   7.   1.   7.   1.  15.   3.   5.  11.   6.   1.  11.  17.   6.   6.
   7.  10.   7.  15.   8.   7.  19.   3.   3.  12.  12.   8.  14.   7.   6.
   6.   5.  19.  11.   8.   6.  10.   3.   6.   6.   8.   6.  15.   8.  12.
  14.   7.   2.  17.   7.   0.   7.  10.   6.   6.   6.   3.  10.  13.   3.
  15.   3.   7.   1.   0.   8.   5.   6.  17.   9.   1.   0.   6.  12.   8.
   3.   6.   7.   8.  18.   1.  14.   0.   6.   3.]


In [103]:
Y_answers = []

for i in range(len(Y_pred)):
    Y_answers.append(list_cuisines[int(Y_pred[i])])
    if i % 2000 == 0:
        print i
        
print "list complete."

0
2000
4000
6000
8000
list complete.


In [96]:
print Y_answers

[u'british', u'southern_us', u'italian', u'cajun_creole', u'italian', u'southern_us', u'spanish', u'chinese', u'mexican', u'british', u'italian', u'greek', u'indian', u'italian', u'southern_us', u'french', u'mexican', u'southern_us', u'mexican', u'southern_us', u'japanese', u'indian', u'spanish', u'vietnamese', u'italian', u'southern_us', u'vietnamese', u'korean', u'italian', u'italian', u'mexican', u'thai', u'mexican', u'japanese', u'chinese', u'mexican', u'russian', u'indian', u'indian', u'cajun_creole', u'cajun_creole', u'chinese', u'french', u'mexican', u'italian', u'italian', u'spanish', u'russian', u'vietnamese', u'chinese', u'italian', u'thai', u'indian', u'italian', u'italian', u'chinese', u'italian', u'japanese', u'chinese', u'cajun_creole', u'french', u'mexican', u'filipino', u'korean', u'mexican', u'greek', u'mexican', u'thai', u'italian', u'italian', u'italian', u'indian', u'thai', u'brazilian', u'indian', u'japanese', u'indian', u'mexican', u'southern_us', u'greek', u'chin

In [138]:
Y_predicted = np.asarray(Y_answers)
y = np.vstack((df_test['id'], Y_predicted)).transpose()
print y

[[u'18009' u'british']
 [u'28583' u'southern_us']
 [u'41580' u'italian']
 ..., 
 [u'22339' u'italian']
 [u'42525' u'southern_us']
 [u'1443' u'mexican']]


In [141]:
np.savetxt('predictions.csv', y, fmt="%s", delimiter=',', header='id,cuisine', comments='')

print "csv file complete."

csv file complete.


In [ ]:
# Accuracy at 0.78319